# Duplicate Application Dataset: Load, Canonicalize, and Generate Duplicates

This notebook:
- Loads the BPI Challenge 2012 XES log
- Performs initial EDA for application events (A_*)
- Builds canonical datasets (events.csv, applications.csv)
- Provides a generator to synthesize duplicate applications of multiple types
- Writes duplicate samples to data/processed/duplicate_samples.csv
- Optionally builds a master duplicate-only dataset for downstream testing


In [1]:
# Imports and setup
import pm4py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re

plt.style.use('default')
sns.set_palette('husl')

XES_PATH = 'data/BPI_Challenge_2012.xes'


In [24]:
# Create canonical events.csv and applications.csv from XES, including revenue/loan value
import pm4py
import pandas as pd
import numpy as np
from pathlib import Path
import re

# Paths
XES_PATH = 'data/BPI_Challenge_2012.xes'
OUT_DIR = Path('data/processed')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Load XES → events (raw)
print('Loading XES:', XES_PATH)
log = pm4py.read_xes(XES_PATH)
events = pm4py.convert_to_dataframe(log)

# 2) Build applications view (A_* activities only)
apps_df = events[events['concept:name'].str.startswith('A_', na=False)].copy()
apps_df = apps_df[['case:concept:name','concept:name','org:resource','time:timestamp']].rename(columns={
    'case:concept:name': 'application_id',
    'concept:name': 'activity',
    'org:resource': 'user',
    'time:timestamp': 'timestamp'
})

# 2b) Add revenue/loan value if present
# Try common column names, fallback to NaN if not found
value_col = None
for col in ['case:AMOUNT_REQ', 'case:loan_amount', 'case:revenue', 'case:AMOUNT', 'AMOUNT_REQ', 'loan_value', 'revenue']:
    if col in events.columns:
        value_col = col
        break

if value_col:
    apps_df['value'] = events.loc[apps_df.index, value_col]
else:
    apps_df['value'] = np.nan

# 3) Normalize types and timestamps
def to_naive_dt(s):
    s = pd.to_datetime(s, errors='coerce')
    try:
        s = s.dt.tz_convert(None)
    except Exception:
        try:
            s = s.dt.tz_localize(None)
        except Exception:
            pass
    return s

events['time:timestamp'] = to_naive_dt(events['time:timestamp'])
if 'case:REG_DATE' in events.columns:
    events['case:REG_DATE'] = to_naive_dt(events['case:REG_DATE'])

apps_df['timestamp'] = to_naive_dt(apps_df['timestamp'])

# 4) Applications case-level aggregates (include value)
def normalize_app_number(x: str) -> str:
    if pd.isna(x): return ''
    s = str(x)
    s = re.sub(r'^(APP|SYS|WEB|MOB|REF|REQ)', '', s, flags=re.IGNORECASE)
    s = re.sub(r'^0+', '', s)
    return s.lower()

agg = (
    apps_df.sort_values('timestamp')
           .groupby('application_id', as_index=False)
           .agg(
               first_timestamp=('timestamp','first'),
               last_timestamp=('timestamp','last'),
               primary_user=('user','first'),
               n_events=('activity','size'),
               activity_sequence=('activity', lambda a: ' '.join(a.tolist())),
               value=('value','first')  # take first value per application
           )
)
agg['application_number'] = agg['application_id'].astype(str)
agg['app_number_normalized'] = agg['application_number'].apply(normalize_app_number)

applications = agg[['application_id','application_number','app_number_normalized',
                    'first_timestamp','last_timestamp','primary_user','n_events','activity_sequence','value']]

# 5) Sanitize for CSV (strings for objects)
def sanitize_for_csv(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == 'object':
            out[c] = out[c].astype(str)
    return out

events_out = sanitize_for_csv(events)
applications_out = sanitize_for_csv(applications)

# 6) Save CSVs
events_csv = OUT_DIR / 'events.csv'
applications_csv = OUT_DIR / 'applications.csv'
events_out.to_csv(events_csv, index=False)
applications_out.to_csv(applications_csv, index=False)

print('Saved:')
print(' -', events_csv.resolve())
print(' -', applications_csv.resolve())

Loading XES: data/BPI_Challenge_2012.xes


parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]

Saved:
 - /Users/u.srinivasan/Documents/Projects_Garage/Duplicate-Invoice-Solution/data/processed/events.csv
 - /Users/u.srinivasan/Documents/Projects_Garage/Duplicate-Invoice-Solution/data/processed/applications.csv


In [25]:
# Generate a scalable, comprehensive duplicate-applications dataset (all types) and save CSV
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta
import random
import re
from rapidfuzz import fuzz

# Config
INPUT_APPS_CSV = 'data/processed/applications.csv'
OUT_DIR = Path('data/processed')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Size controls
target_total = 50000
seed = 42

random.seed(seed)
np.random.seed(seed)

apps = pd.read_csv(
    INPUT_APPS_CSV,
    parse_dates=['first_timestamp','last_timestamp'],
    dtype={'application_id': str, 'application_number': str, 'app_number_normalized': str}
)

# Ensure amount_req column exists (fallback to value if not)
if 'amount_req' not in apps.columns:
    if 'value' in apps.columns:
        apps['amount_req'] = apps['value']
    else:
        apps['amount_req'] = np.nan

# Helper generators (fuzzy)
def typo_variation(s: str) -> str:
    if not s: return s
    i = random.randrange(len(s))
    ch = s[i]
    if ch.isdigit():
        repl = str((int(ch) + random.randint(1, 9)) % 10)
    else:
        repl = chr(((ord(ch.lower()) - 97 + random.randint(1, 3)) % 26) + 97)
    return s[:i] + repl + s[i+1:]

def insert_or_delete(s: str) -> str:
    if not s: return s
    if random.random() < 0.5 and len(s) < 18:
        i = random.randrange(len(s)+1)
        add = random.choice('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        return s[:i] + add + s[i:]
    if len(s) > 1:
        i = random.randrange(len(s))
        return s[:i] + s[i+1:]
    return s

def transpose_adjacent(s: str) -> str:
    if len(s) < 2: return s
    i = random.randrange(len(s)-1)
    t = list(s); t[i], t[i+1] = t[i+1], t[i]
    return ''.join(t)

# Helper generators (non-fuzzy/semantic)
def prefix_format(s: str) -> str:
    pref = random.choice(['APP','SYS','WEB','MOB','REF','REQ'])
    return f'{pref}{s}'

def zero_pad(s: str, width: int = 10) -> str:
    digits = ''.join([c for c in s if c.isdigit()]) or s
    try:
        return str(int(digits)).zfill(width)
    except Exception:
        return digits.zfill(width)

def normalize(s: str) -> str:
    s = re.sub(r'^(APP|SYS|WEB|MOB|REF|REQ)', '', str(s), flags=re.IGNORECASE)
    s = re.sub(r'^0+', '', s)
    return s.lower()

def time_shift(dt, minutes_low, minutes_high):
    if pd.isna(dt): dt = pd.Timestamp.now()
    return dt + timedelta(minutes=random.randint(minutes_low, minutes_high))

def pick_alt_user(primary_user: str, pool) -> str:
    choices = [u for u in pool if u != str(primary_user)]
    return random.choice(choices or pool)

def amount_variation(amount):
    # Add or subtract up to 5% for fuzzy, or up to 20% for non-fuzzy
    if pd.isna(amount): return amount
    if random.random() < 0.5:
        return amount * (1 + random.uniform(-0.05, 0.05))  # fuzzy
    else:
        return amount * (1 + random.uniform(-0.2, 0.2))    # non-fuzzy

def make_row(a, dup_num, dup_type, variation, user_b=None, ts_b=None, act_b=None, amt_b=None):
    a_num = a['application_number']
    ft_a = a['first_timestamp']
    if ts_b is None:
        ts_b = time_shift(ft_a, 5, 24*60)
    if user_b is None:
        user_b = a.get('primary_user', '')
    if act_b is None:
        act_b = a.get('activity_sequence', '')
    if amt_b is None:
        amt_b = amount_variation(a.get('amount_req', np.nan))
    return {
        'original_application_id': a['application_id'],
        'original_application_number': a_num,
        'duplicate_application_number': dup_num,
        'duplicate_type': dup_type,
        'variation': variation,
        'expected_detection': 'fuzzy_detectable' if dup_type.startswith('fuzzy') else 'non_fuzzy_detectable',
        'primary_user_A': str(a.get('primary_user','')),
        'primary_user_B': str(user_b),
        'first_timestamp_A': ft_a,
        'first_timestamp_B': ts_b,
        'activity_sequence_A': a.get('activity_sequence',''),
        'activity_sequence_B': act_b,
        'amount_req_A': a.get('amount_req', np.nan),
        'amount_req_B': amt_b,
        'string_similarity': fuzz.ratio(str(a_num), str(dup_num))
    }

def generate_dups(apps_df: pd.DataFrame, count_per_type: int) -> pd.DataFrame:
    rows = []
    base = apps_df.sample(n=min(len(apps_df), count_per_type*6), random_state=seed).reset_index(drop=True)
    users_pool = apps_df['primary_user'].dropna().astype(str).unique().tolist() or ['UserA','UserB','UserC']

    # 1) Fuzzy: typo
    for _, a in base.head(count_per_type).iterrows():
        rows.append(make_row(a, typo_variation(a['application_number']), 'fuzzy_typo', 'typo'))

    # 2) Fuzzy: insert/delete
    for _, a in base.iloc[count_per_type:count_per_type*2].iterrows():
        rows.append(make_row(a, insert_or_delete(a['application_number']), 'fuzzy_insert_delete', 'insert_or_delete'))

    # 3) Fuzzy: transpose
    for _, a in base.iloc[count_per_type*2:count_per_type*3].iterrows():
        rows.append(make_row(a, transpose_adjacent(a['application_number']), 'fuzzy_transpose', 'transpose'))

    # 4) Non-fuzzy: prefix/format
    for _, a in base.iloc[count_per_type*3:count_per_type*4].iterrows():
        rows.append(make_row(a, prefix_format(a['application_number']), 'nonfuzzy_prefix', 'prefix'))

    # 5) Non-fuzzy: zero padding / re-encoding
    for _, a in base.iloc[count_per_type*4:count_per_type*5].iterrows():
        rows.append(make_row(a, zero_pad(a['application_number']), 'nonfuzzy_zero_pad', 'zero_pad'))

    # 6) Non-fuzzy: semantic (alt user + similar time window, keep activities, vary amount)
    for _, a in base.iloc[count_per_type*5:count_per_type*6].iterrows():
        alt_user = pick_alt_user(a.get('primary_user',''), users_pool)
        ts_b = time_shift(a.get('first_timestamp'), 10, 6*60)
        dup_num = prefix_format(zero_pad(a['application_number']))
        amt_b = amount_variation(a.get('amount_req', np.nan))
        rows.append(make_row(a, dup_num, 'nonfuzzy_semantic', 'user_time_window', user_b=alt_user, ts_b=ts_b, amt_b=amt_b))

    dup_df = pd.DataFrame(rows)
    dup_df = dup_df.drop_duplicates(
        subset=['original_application_number','duplicate_application_number'], keep='last'
    ).reset_index(drop=True)
    return dup_df

per_type = max(1, target_total // 6)

shards = []
remaining = target_total
while remaining > 0:
    n = min(per_type, remaining // 6 if remaining >= 6 else 1)
    if n == 0: break
    shards.append(generate_dups(apps, n))
    remaining -= len(shards[-1])

dup_master = pd.concat(shards, ignore_index=True)
dup_master = dup_master.sample(frac=1.0, random_state=seed).reset_index(drop=True)

print('Rows per duplicate type:')
print(dup_master['duplicate_type'].value_counts())

out_csv = OUT_DIR / 'duplicate_applications_master.csv'
dup_master.to_csv(out_csv, index=False)
print('\nSaved:', out_csv.resolve())
print('Total rows:', len(dup_master))
dup_master.head(10)

Rows per duplicate type:
duplicate_type
fuzzy_typo             20246
fuzzy_insert_delete    16667
fuzzy_transpose         6544
nonfuzzy_prefix         2964
nonfuzzy_semantic       1790
nonfuzzy_zero_pad       1790
Name: count, dtype: int64

Saved: /Users/u.srinivasan/Documents/Projects_Garage/Duplicate-Invoice-Solution/data/processed/duplicate_applications_master.csv
Total rows: 50001


,original_application_id,original_application_number,duplicate_application_number,duplicate_type,variation,expected_detection,primary_user_A,primary_user_B,first_timestamp_A,first_timestamp_B,activity_sequence_A,activity_sequence_B,amount_req_A,amount_req_B,string_similarity
0,188371,188371,1883711,fuzzy_insert_delete,insert_or_delete,fuzzy_detectable,112,112,2011-11-23 10:42:16.720,2011-11-23 21:14:16.720,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_CANCELLED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_CANCELLED,30000,33246.151436,92.307692
1,196048,196048,R196048,fuzzy_insert_delete,insert_or_delete,fuzzy_detectable,112,112,2011-12-28 13:05:49.406,2011-12-28 13:36:49.406,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,300,295.890162,92.307692
2,204604,204604,208604,fuzzy_typo,typo,fuzzy_detectable,112,112,2012-01-28 13:30:36.915,2012-01-28 16:40:36.915,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_DECLINED,5000,4799.221563,83.333333
3,185437,185437,18543X7,fuzzy_insert_delete,insert_or_delete,fuzzy_detectable,112,112,2011-11-15 09:00:34.480,2011-11-15 18:07:34.480,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,50000,50389.561158,92.307692
4,177092,177092,137092,fuzzy_typo,typo,fuzzy_detectable,112,112,2011-10-14 22:19:53.123,2011-10-15 06:46:53.123,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_CANCELLED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_CANCELLED,10000,10431.074865,83.333333
5,188410,188410,18410,fuzzy_insert_delete,insert_or_delete,fuzzy_detectable,112,112,2011-11-23 12:44:02.815,2011-11-23 21:59:02.815,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,5000,4931.983234,90.909091
6,204496,204496,24496,fuzzy_insert_delete,insert_or_delete,fuzzy_detectable,112,112,2012-01-27 17:47:59.852,2012-01-28 12:56:59.852,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED,30000,25269.416924,90.909091
7,207322,207322,SYS0000207322,nonfuzzy_semantic,user_time_window,non_fuzzy_detectable,112,112,2012-02-07 22:14:39.816,2012-02-07 23:04:39.816,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,2000,1904.926355,63.157895
8,175302,175302,115302,fuzzy_typo,typo,fuzzy_detectable,112,112,2011-10-07 11:46:03.522,2011-10-07 19:14:03.522,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_APPROVED A_REGISTERED A_ACTIVATED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_APPROVED A_REGISTERED A_ACTIVATED,20000,23859.772930,83.333333
9,204400,204400,204400,fuzzy_transpose,transpose,fuzzy_detectable,112,112,2012-01-27 10:46:55.693,2012-01-28 08:15:55.693,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_CANCELLED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_CANCELLED,17500,17993.136069,100.000000


In [26]:
# Convert duplicate_applications_master.csv → applications-like structure and save (with amount_req)
import pandas as pd
import re
from pathlib import Path
from datetime import timedelta

IN_DUP = 'data/processed/duplicate_applications_master.csv'   # from previous step
OUT_DIR = Path('data/processed')
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_CSV = OUT_DIR / 'applications_duplicates.csv'

# Load
dup = pd.read_csv(
    IN_DUP,
    parse_dates=['first_timestamp_A','first_timestamp_B'],
    dtype={'original_application_id': str,
           'original_application_number': str,
           'duplicate_application_number': str}
)

# Helper: normalize application number (same rule used for applications.csv)
def normalize_app_number(x: str) -> str:
    if pd.isna(x): return ''
    s = str(x)
    s = re.sub(r'^(APP|SYS|WEB|MOB|REF|REQ)', '', s, flags=re.IGNORECASE)
    s = re.sub(r'^0+', '', s)
    return s.lower()

# Derive required columns to match applications.csv schema:
# ['application_id','application_number','app_number_normalized',
#  'first_timestamp','last_timestamp','primary_user','n_events','activity_sequence','amount_req']
out = pd.DataFrame()

# Use the duplicate_application_number as both application_id and application_number (string)
out['application_id'] = dup['duplicate_application_number'].astype(str)
out['application_number'] = dup['duplicate_application_number'].astype(str)
out['app_number_normalized'] = out['application_number'].apply(normalize_app_number)

# Timestamps: use first_timestamp_B; synthesize last_timestamp by adding 5 minutes per event
out['first_timestamp'] = dup['first_timestamp_B']

# n_events from activity_sequence_B (fallback to 1 if empty)
seq_b = dup['activity_sequence_B'].fillna('').astype(str)
n_events = seq_b.apply(lambda s: max(1, len([t for t in s.split(' ') if t])))
out['n_events'] = n_events

# last_timestamp = first_timestamp + 5 minutes per event (minimum +5 minutes)
duration_minutes_per_event = 5
out['last_timestamp'] = out['first_timestamp'] + pd.to_timedelta(out['n_events'] * duration_minutes_per_event, unit='m')

# primary_user from B
out['primary_user'] = dup['primary_user_B'].astype(str)

# activity_sequence from B
out['activity_sequence'] = seq_b

# amount_req from B (if present, else fallback to NaN)
if 'amount_req_B' in dup.columns:
    out['amount_req'] = dup['amount_req_B']
elif 'value_B' in dup.columns:
    out['amount_req'] = dup['value_B']
else:
    out['amount_req'] = np.nan

# Optional: ensure column order exactly matches applications.csv (+ amount_req)
out = out[['application_id','application_number','app_number_normalized',
           'first_timestamp','last_timestamp','primary_user','n_events','activity_sequence','amount_req']]

# Save
out.to_csv(OUT_CSV, index=False)
print('Saved:', OUT_CSV.resolve())
print('Rows:', len(out))
out.head(5)

Saved: /Users/u.srinivasan/Documents/Projects_Garage/Duplicate-Invoice-Solution/data/processed/applications_duplicates.csv
Rows: 50001


,application_id,application_number,app_number_normalized,first_timestamp,last_timestamp,primary_user,n_events,activity_sequence,amount_req
0,1883711,1883711,1883711,2011-11-23 21:14:16.720,2011-11-23 21:34:16.720,112,4,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_CANCELLED,33246.151436
1,R196048,R196048,r196048,2011-12-28 13:36:49.406,2011-12-28 13:51:49.406,112,3,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,295.890162
2,208604,208604,208604,2012-01-28 16:40:36.915,2012-01-28 17:10:36.915,112,6,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_DECLINED,4799.221563
3,18543X7,18543X7,18543x7,2011-11-15 18:07:34.480,2011-11-15 18:22:34.480,112,3,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,50389.561158
4,137092,137092,137092,2011-10-15 06:46:53.123,2011-10-15 07:16:53.123,112,6,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_CANCELLED,10431.074865


In [27]:
# Config and data load (run once)
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta
import re
from rapidfuzz import fuzz

pd.set_option('display.max_colwidth', 120)

DATA_DIR = Path('data/processed')
APPS = DATA_DIR / 'applications.csv'
APPS_DUP = DATA_DIR / 'applications_duplicates.csv'  # treat as “unknown” new applications

apps = pd.read_csv(
    APPS,
    parse_dates=['first_timestamp','last_timestamp'],
    dtype={'application_id': str, 'application_number': str, 'app_number_normalized': str}
)

# Ensure amount_req column exists (fallback to value if not)
if 'amount_req' not in apps.columns:
    if 'value' in apps.columns:
        apps['amount_req'] = apps['value']
    else:
        apps['amount_req'] = np.nan

cand = pd.read_csv(
    APPS_DUP,
    parse_dates=['first_timestamp','last_timestamp'],
    dtype={'application_id': str, 'application_number': str, 'app_number_normalized': str}
)

# Ensure amount_req column exists (fallback to value if not)
if 'amount_req' not in cand.columns:
    if 'value' in cand.columns:
        cand['amount_req'] = cand['value']
    else:
        cand['amount_req'] = np.nan

print('Loaded:')
print(' - apps:', len(apps))
print(' - candidates:', len(cand))
print(' - apps amount_req present:', 'amount_req' in apps.columns)
print(' - candidates amount_req present:', 'amount_req' in cand.columns)

Loaded:
 - apps: 13087
 - candidates: 50001
 - apps amount_req present: True
 - candidates amount_req present: True


1) Data normalization and canonicalization

In [28]:
# Step 1: Normalize keys and build canonical fields (with amount_req)

def normalize_app_number(x: str) -> str:
    if pd.isna(x): return ''
    s = str(x)
    s = re.sub(r'^(APP|SYS|WEB|MOB|REF|REQ)', '', s, flags=re.IGNORECASE)
    s = re.sub(r'^0+', '', s)
    return s.lower()

def prep(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['application_number'] = out['application_number'].astype(str)
    out['app_number_norm_v2'] = out['application_number'].apply(normalize_app_number)
    out['primary_user'] = out['primary_user'].astype(str)
    out['first_timestamp'] = pd.to_datetime(out['first_timestamp'], errors='coerce')
    out['last_timestamp'] = pd.to_datetime(out['last_timestamp'], errors='coerce')
    # simple activity signature
    out['activity_seq'] = out['activity_sequence'].fillna('').astype(str)
    out['activity_sig_3'] = out['activity_seq'].apply(lambda s: ' '.join(s.split()[:3]))
    out['activity_len'] = out['activity_seq'].apply(lambda s: max(1, len(s.split())))
    # Ensure amount_req exists
    if 'amount_req' not in out.columns:
        if 'value' in out.columns:
            out['amount_req'] = out['value']
        else:
            out['amount_req'] = np.nan
    return out

apps1 = prep(apps)
cand1 = prep(cand)

# Sample results
display(apps1.head(3)[['application_number','app_number_norm_v2','primary_user','first_timestamp','activity_sig_3','amount_req']])
display(cand1.head(3)[['application_number','app_number_norm_v2','primary_user','first_timestamp','activity_sig_3','amount_req']])

# --- Enhanced duplicate reasoning using amount_req ---
from rapidfuzz import fuzz

def strict_duplicate_reason(row, time_tolerance_hours=1, activity_similarity_threshold=80, amount_tolerance=0.01):
    user_match = row['primary_user_cand'] == row['primary_user_apps']
    time_diff = abs((row['first_timestamp_cand'] - row['first_timestamp_apps']).total_seconds()) / 3600
    time_match = time_diff <= time_tolerance_hours
    activity_sim = fuzz.ratio(str(row.get('activity_sequence_cand', '')), str(row.get('activity_sequence_apps', '')))
    activity_match = activity_sim >= activity_similarity_threshold
    # Amount match: within tolerance (default 1%)
    amt_a = row.get('amount_req_cand', np.nan)
    amt_b = row.get('amount_req_apps', np.nan)
    if pd.notna(amt_a) and pd.notna(amt_b) and amt_a > 0:
        amount_match = abs(amt_a - amt_b) / amt_a <= amount_tolerance
    else:
        amount_match = False
    reasons = []
    reasons.append("Primary user matches" if user_match else "Primary user does not match")
    reasons.append(f"Timestamp within {time_tolerance_hours}h ({time_diff:.2f}h)" if time_match else f"Timestamp outside {time_tolerance_hours}h ({time_diff:.2f}h)")
    reasons.append(f"Activity sequence similarity {activity_sim} ≥ {activity_similarity_threshold}" if activity_match else f"Activity sequence similarity {activity_sim} < {activity_similarity_threshold}")
    reasons.append("Amount matches" if amount_match else "Amount does not match")
    return "; ".join(reasons)

def false_positive_reason(row, activity_similarity_threshold=30, amount_tolerance=0.05):
    user_mismatch = row['primary_user_cand'] != row['primary_user_apps']
    activity_sim = fuzz.ratio(str(row.get('activity_sequence_cand', '')), str(row.get('activity_sequence_apps', '')))
    activity_low = activity_sim < activity_similarity_threshold
    amt_a = row.get('amount_req_cand', np.nan)
    amt_b = row.get('amount_req_apps', np.nan)
    if pd.notna(amt_a) and pd.notna(amt_b) and amt_a > 0:
        amount_mismatch = abs(amt_a - amt_b) / amt_a > amount_tolerance
    else:
        amount_mismatch = True
    reasons = []
    reasons.append("Primary user does not match" if user_mismatch else "Primary user matches")
    reasons.append(f"Activity sequence similarity {activity_sim} < {activity_similarity_threshold}" if activity_low else f"Activity sequence similarity {activity_sim} ≥ {activity_similarity_threshold}")
    reasons.append("Amount does not match" if amount_mismatch else "Amount matches")
    return "; ".join(reasons)

# When merging/joining, ensure amount_req columns are included for both cand1 and apps1
# Example join for reasoning:
# root_matches_full = root_matches.merge(
#     cand1[['application_id', 'activity_sequence', 'amount_req']].rename(columns={'application_id': 'application_id_cand', 'activity_sequence': 'activity_sequence_cand', 'amount_req': 'amount_req_cand'}),
#     on='application_id_cand', how='left'
# ).merge(
#     apps1[['application_id', 'activity_sequence', 'amount_req']].rename(columns={'application_id': 'application_id_apps', 'activity_sequence': 'activity_sequence_apps', 'amount_req': 'amount_req_apps'}),
#     on='application_id_apps', how='left'
# )

# Now you can use strict_duplicate_reason and false_positive_reason for better

,application_number,app_number_norm_v2,primary_user,first_timestamp,activity_sig_3,amount_req
0,173688,173688,112,2011-10-01 00:38:44.546,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED,20000
1,173691,173691,112,2011-10-01 08:08:58.256,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED,5000
2,173694,173694,112,2011-10-01 08:10:30.287,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED,7000


,application_number,app_number_norm_v2,primary_user,first_timestamp,activity_sig_3,amount_req
0,1883711,1883711,112,2011-11-23 21:14:16.720,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED,33246.151436
1,R196048,r196048,112,2011-12-28 13:36:49.406,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,295.890162
2,208604,208604,112,2012-01-28 16:40:36.915,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED,4799.221563


In [29]:
# Merge activity_sequence and amount_req columns for comparison
root_matches_full = root_matches.merge(
    cand1[['application_id', 'activity_sequence', 'amount_req']].rename(columns={
        'application_id': 'application_id_cand',
        'activity_sequence': 'activity_sequence_cand',
        'amount_req': 'amount_req_cand'
    }),
    on='application_id_cand', how='left'
).merge(
    apps1[['application_id', 'activity_sequence', 'amount_req']].rename(columns={
        'application_id': 'application_id_apps',
        'activity_sequence': 'activity_sequence_apps',
        'amount_req': 'amount_req_apps'
    }),
    on='application_id_apps', how='left'
)

# Reasoning for strict duplicates (now includes amount_req)
def strict_duplicate_reason(row, time_tolerance_hours=1, activity_similarity_threshold=80, amount_tolerance=0.01):
    from rapidfuzz import fuzz
    user_match = row['primary_user_cand'] == row['primary_user_apps']
    time_diff = abs((row['first_timestamp_cand'] - row['first_timestamp_apps']).total_seconds()) / 3600
    time_match = time_diff <= time_tolerance_hours
    activity_sim = fuzz.ratio(str(row.get('activity_sequence_cand', '')), str(row.get('activity_sequence_apps', '')))
    activity_match = activity_sim >= activity_similarity_threshold
    amt_a = row.get('amount_req_cand', np.nan)
    amt_b = row.get('amount_req_apps', np.nan)
    if pd.notna(amt_a) and pd.notna(amt_b) and amt_a > 0:
        amount_match = abs(amt_a - amt_b) / amt_a <= amount_tolerance
    else:
        amount_match = False
    reasons = []
    reasons.append("Primary user matches" if user_match else "Primary user does not match")
    reasons.append(f"Timestamp within {time_tolerance_hours} hour(s) ({time_diff:.2f}h)" if time_match else f"Timestamp outside {time_tolerance_hours} hour(s) ({time_diff:.2f}h)")
    reasons.append(f"Activity sequence similarity {activity_sim} ≥ {activity_similarity_threshold}" if activity_match else f"Activity sequence similarity {activity_sim} < {activity_similarity_threshold}")
    reasons.append("Amount matches" if amount_match else "Amount does not match")
    return "; ".join(reasons)

# Reasoning for false positives (now includes amount_req)
def false_positive_reason(row, activity_similarity_threshold=30, amount_tolerance=0.05):
    from rapidfuzz import fuzz
    user_mismatch = row['primary_user_cand'] != row['primary_user_apps']
    activity_sim = fuzz.ratio(str(row.get('activity_sequence_cand', '')), str(row.get('activity_sequence_apps', '')))
    activity_low = activity_sim < activity_similarity_threshold
    amt_a = row.get('amount_req_cand', np.nan)
    amt_b = row.get('amount_req_apps', np.nan)
    if pd.notna(amt_a) and pd.notna(amt_b) and amt_a > 0:
        amount_mismatch = abs(amt_a - amt_b) / amt_a > amount_tolerance
    else:
        amount_mismatch = True
    reasons = []
    reasons.append("Primary user does not match" if user_mismatch else "Primary user matches")
    reasons.append(f"Activity sequence similarity {activity_sim} < {activity_similarity_threshold}" if activity_low else f"Activity sequence similarity {activity_sim} ≥ {activity_similarity_threshold}")
    reasons.append("Amount does not match" if amount_mismatch else "Amount matches")
    return "; ".join(reasons)

# Apply strict duplicate check and reasoning (with amount_req)
root_matches_full['is_strict_duplicate'] = root_matches_full.apply(
    lambda row: (
        row['primary_user_cand'] == row['primary_user_apps'] and
        abs((row['first_timestamp_cand'] - row['first_timestamp_apps']).total_seconds()) / 3600 <= 1 and
        __import__('rapidfuzz').fuzz.ratio(str(row.get('activity_sequence_cand', '')), str(row.get('activity_sequence_apps', ''))) >= 80 and
        pd.notna(row.get('amount_req_cand', np.nan)) and pd.notna(row.get('amount_req_apps', np.nan)) and row.get('amount_req_cand', 0) > 0 and
        abs(row.get('amount_req_cand', 0) - row.get('amount_req_apps', 0)) / row.get('amount_req_cand', 1) <= 0.01
    ), axis=1
)
root_matches_full['strict_duplicate_reason'] = root_matches_full.apply(
    lambda row: strict_duplicate_reason(row, time_tolerance_hours=1, activity_similarity_threshold=80, amount_tolerance=0.01), axis=1
)

strict_duplicates = root_matches_full[root_matches_full['is_strict_duplicate']].copy()

print('Strict duplicates after additional checks:', len(strict_duplicates))
display(strict_duplicates.head(5)[[
    'application_number_cand','application_number_apps','app_number_norm_v2',
    'primary_user_cand','primary_user_apps','first_timestamp_cand','first_timestamp_apps',
    'activity_sequence_cand','activity_sequence_apps','amount_req_cand','amount_req_apps','strict_duplicate_reason'
]])

# Apply false positive check and reasoning (with amount_req)
root_matches_full['is_false_positive'] = root_matches_full.apply(
    lambda row: (
        row['primary_user_cand'] != row['primary_user_apps'] and
        __import__('rapidfuzz').fuzz.ratio(str(row.get('activity_sequence_cand', '')), str(row.get('activity_sequence_apps', ''))) < 30 and
        pd.notna(row.get('amount_req_cand', np.nan)) and pd.notna(row.get('amount_req_apps', np.nan)) and row.get('amount_req_cand', 0) > 0 and
        abs(row.get('amount_req_cand', 0) - row.get('amount_req_apps', 0)) / row.get('amount_req_cand', 1) > 0.05
    ), axis=1
)
root_matches_full['false_positive_reason'] = root_matches_full.apply(
    lambda row: false_positive_reason(row, activity_similarity_threshold=30, amount_tolerance=0.05), axis=1
)

false_positives = root_matches_full[root_matches_full['is_false_positive']].copy()

print('Definite false positives:', len(false_positives))
display(false_positives.head(5)[[
    'application_number_cand','application_number_apps','app_number_norm_v2',
    'primary_user_cand','primary_user_apps','first_timestamp_cand','first_timestamp_apps',
    'activity_sequence_cand','activity_sequence_apps','amount_req_cand','amount_req_apps','false_positive_reason'
]])

# Remove all applications involved in strict duplicates or false positives from further checks
exclude_cand_ids = set(strict_duplicates['application_id_cand']).union(false_positives['application_id_cand'])
exclude_apps_ids = set(strict_duplicates['application_id_apps']).union(false_positives['application_id_apps'])

maybe_cand1 = cand1[~cand1['application_id'].isin(exclude_cand_ids)].copy()
maybe_apps1 = apps1[~apps1['application_id'].isin(exclude_apps_ids)].copy()

print('Remaining candidates for further checks:', len(maybe_cand1))
print('Remaining apps for further checks:', len(maybe_apps1))

print(f"\nSummary:")
print(f"True duplicates identified: {len(strict_duplicates)}")
print(f"False positives identified: {len(false_positives)}")
print(f"Remaining maybe duplicate applications (candidates): {len(maybe_cand1)}")
print(f"Remaining maybe duplicate applications (apps): {len(maybe_apps1)}")

Strict duplicates after additional checks: 20


,application_number_cand,application_number_apps,app_number_norm_v2,primary_user_cand,primary_user_apps,first_timestamp_cand,first_timestamp_apps,activity_sequence_cand,activity_sequence_apps,amount_req_cand,amount_req_apps,strict_duplicate_reason
11,REQ0000200308,200308,200308,112,112,2012-01-12 17:29:58.356,2012-01-12 16:54:58.356,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,9962.049545,10000,Primary user matches; Timestamp within 1 hour(s) (0.58h); Activity sequence similarity 100.0 ≥ 80; Amount matches
1617,0000190854,190854,190854,112,112,2011-12-03 00:17:58.201,2011-12-03 00:12:58.201,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,2486.092171,2500,Primary user matches; Timestamp within 1 hour(s) (0.08h); Activity sequence similarity 100.0 ≥ 80; Amount matches
2066,0000201436,201436,201436,112,112,2012-01-16 19:12:45.783,2012-01-16 18:31:45.783,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_ACCEPTED A_FINALIZED A_DECLINED,4958.371012,5000,Primary user matches; Timestamp within 1 hour(s) (0.68h); Activity sequence similarity 100.0 ≥ 80; Amount matches
2120,0000201623,201623,201623,112,112,2012-01-17 10:55:41.215,2012-01-17 10:40:41.215,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_DECLINED,14963.538587,15000,Primary user matches; Timestamp within 1 hour(s) (0.25h); Activity sequence similarity 100.0 ≥ 80; Amount matches
2831,0000175012,175012,175012,112,112,2011-10-06 12:00:32.215,2011-10-06 11:23:32.215,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_DECLINED,A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED A_DECLINED,49625.019308,50000,Primary user matches; Timestamp within 1 hour(s) (0.62h); Activity sequence similarity 100.0 ≥ 80; Amount matches


Definite false positives: 0


,application_number_cand,application_number_apps,app_number_norm_v2,primary_user_cand,primary_user_apps,first_timestamp_cand,first_timestamp_apps,activity_sequence_cand,activity_sequence_apps,amount_req_cand,amount_req_apps,false_positive_reason


Remaining candidates for further checks: 49980
Remaining apps for further checks: 13067

Summary:
True duplicates identified: 20
False positives identified: 0
Remaining maybe duplicate applications (candidates): 49980
Remaining maybe duplicate applications (apps): 13067


In [40]:
import numpy as np
from rapidfuzz import fuzz

# 1) Pick a random application from the remaining maybe pile
random_app = maybe_cand1.sample(n=1, random_state=72).iloc[0]

# 2) Compute similarity scores with all remaining apps, including amount_req
def compute_similarity(row, ref_app):
    # You can adjust weights as needed
    id_score = fuzz.ratio(str(row['app_number_norm_v2']), str(ref_app['app_number_norm_v2']))
    activity_score = fuzz.ratio(str(row['activity_seq']), str(ref_app['activity_seq']))
    user_score = 100 if row['primary_user'] == ref_app['primary_user'] else 0
    time_score = (1 - min(abs((row['first_timestamp'] - ref_app['first_timestamp']).total_seconds()) / (3600*24), 1)) * 100
    # Amount similarity: 100 if within 1%, else decay linearly to 0 at 20% difference
    amt_a = row.get('amount_req', np.nan)
    amt_b = ref_app.get('amount_req', np.nan)
    if pd.notna(amt_a) and pd.notna(amt_b) and amt_a > 0:
        amt_diff = abs(amt_a - amt_b) / amt_a
        if amt_diff <= 0.01:
            amount_score = 100
        elif amt_diff <= 0.2:
            amount_score = 100 * (1 - (amt_diff - 0.01) / 0.19)
        else:
            amount_score = 0
    else:
        amount_score = 0
    # Weighted sum (tune weights as needed)
    score = (
        0.65 * id_score +
        0.35 * amount_score
    )
    return score

maybe_apps1['similarity_score'] = maybe_apps1.apply(lambda row: compute_similarity(row, random_app), axis=1)

# 3) Get top 20 matches for the selected application (now includes amount_req)
top_matches = maybe_apps1.sort_values('similarity_score', ascending=False).head(50)

print(f"Randomly selected application: {random_app['application_number']}")
print("Top 10 closest matches (by similarity score, including amount_req):")
display(top_matches[['application_number','app_number_norm_v2','primary_user','first_timestamp','amount_req','similarity_score']].head(10))

Randomly selected application: SYS184619
Top 10 closest matches (by similarity score, including amount_req):


,application_number,app_number_norm_v2,primary_user,first_timestamp,amount_req,similarity_score
3503,184619,184619,112,2011-11-11 17:39:19.320,50000,98.956612
3176,183619,183619,112,2011-11-09 22:04:29.897,50000,88.123279
296,174629,174629,112,2011-10-05 10:00:20.578,50000,77.289946
1526,178497,178497,112,2011-10-20 09:56:22.308,50000,77.289946
5045,189406,189406,112,2011-11-27 21:53:26.370,50000,77.289946
2663,182041,182041,112,2011-11-03 18:18:35.150,50000,77.289946
2159,180499,180499,112,2011-10-28 12:11:57.570,50000,77.289946
7985,198561,198561,112,2012-01-06 12:10:22.681,50000,77.289946
3457,184481,184481,112,2011-11-11 11:51:47.118,50000,77.289946
3342,184132,184132,112,2011-11-10 16:52:01.908,50000,77.289946


In [41]:
def refine_similarity(row, ref_app):
    activity_score = fuzz.ratio(str(row['activity_seq']), str(ref_app['activity_seq']))
    user_score = 100 if row['primary_user'] == ref_app['primary_user'] else 0
    time_score = (1 - min(abs((row['first_timestamp'] - ref_app['first_timestamp']).total_seconds()) / (3600*24), 1)) * 100
    # Weighted sum for refinement (adjust weights as needed)
    score = (
        0.4 * activity_score +
        0.3 * user_score +
        0.3 * time_score
    )
    return score

top_matches['refined_similarity'] = top_matches.apply(lambda row: refine_similarity(row, random_app), axis=1)
top_matches_refined = top_matches[top_matches['refined_similarity'] > 80].sort_values('refined_similarity', ascending=False).reset_index(drop=True)

print("Top matches with refined similarity > 80% (user, timestamp, activity sequence):")
display(top_matches_refined[['application_number','app_number_norm_v2','primary_user','first_timestamp','amount_req','similarity_score','refined_similarity']])

Top matches with refined similarity > 80% (user, timestamp, activity sequence):


,application_number,app_number_norm_v2,primary_user,first_timestamp,amount_req,similarity_score,refined_similarity
0,184619,184619,112,2011-11-11 17:39:19.320,50000,98.956612,91.958333
1,184646,184646,112,2011-11-11 18:58:56.114,50000,77.289946,86.699649
2,184601,184601,112,2011-11-11 16:04:45.389,45000,73.747039,83.070925


In [ ]:
# Further filter top_matches_refined using last_timestamp, n_events, and activity_sig_3

# 1. Filter by last_timestamp: within 1 day of the random_app
filtered_by_last_ts = top_matches_refined[
    (abs((top_matches_refined['last_timestamp'] - random_app['last_timestamp']).dt.total_seconds()) <= 3600*24)
]

# 2. Filter by n_events: difference no more than 1
filtered_by_n_events = filtered_by_last_ts[
    (abs(filtered_by_last_ts['n_events'] - random_app['n_events']) <= 2)
]

# 3. Filter by activity_sig_3: must match exactly
filtered_final = filtered_by_n_events[
    (filtered_by_n_events['activity_sig_3'] == random_app['activity_sig_3'])
].reset_index(drop=True)

print("Final filtered matches (last_timestamp, n_events, activity_sig_3):")
display(filtered_final[['application_number','app_number_norm_v2','primary_user','first_timestamp','last_timestamp','n_events','activity_sig_3','amount_req','similarity_score','refined_similarity']])

Final filtered matches (last_timestamp, n_events):


,application_number,app_number_norm_v2,primary_user,first_timestamp,last_timestamp,n_events,amount_req,similarity_score,refined_similarity
